# Talk with your Database


Databases are an ineavitable part of every company's infrastructure. Chatbots capable of interacting with databases can free up teams' time by handling novel user queries.

In this tutorial, we will build an agent with access to the database tool, being able to ground its answers with data stored there. Along the way we will create:
1. Custom LangChain tool.
2. Assistant agent with access to database tool.
3. Tool agent, specialized in executing calls returned by an assistant.
4. Graph of connected agents.
5. Persistent storage component.

By the end, you'll be able to mix this simple strategy with other even more powerful LangGraph concepts.


## Prerequisites


First, set up your environment. We'll install this tutorial's prerequisites, download the test DB, and define the tools we will reuse in each section.

In [ ]:
!pip install -U langgraph langchain langchain_openai langchain_experimental dbally[openai,langsmith] nest_asyncio

In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

### Populate the database

Here, we just fill a dummy database containing some fictional HR information.

In [ ]:
from urllib import request
from sqlalchemy import create_engine, text

print("Downloading the HR database")
request.urlretrieve(
    "https://drive.google.com/uc?export=download&id=1zo3j8x7qH8opTKyQ9qFgRpS3yqU6uTRs", "recruitment.db"
)
print("Database downloaded")
print("Creating the database")

db_engine = create_engine("sqlite:///recruitment.db")

print("Displaying the first 5 rows of the candidate table")

with db_engine.connect() as conn:
    rows = conn.execute(text("SELECT * from candidate LIMIT 5")).fetchall()

print(rows)

## Database Tool

Next, define our [assistant database tool](https://python.langchain.com/v0.1/docs/modules/tools/) to help it answer any questions concerning HR. Under the hood, it uses [db-ally](https://github.com/deepsense-ai/db-ally) database framework. 

In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import Optional, Type
from langchain.callbacks.manager import CallbackManagerForToolRun
from langchain.tools import BaseTool

from dbally import Collection
from dbally.utils.errors import UnsupportedQueryError

import asyncio
import nest_asyncio

nest_asyncio.apply()


class DatabaseQuery(BaseModel):
    query: str = Field(description="should be a query to the database in the natural language.")


class DballyTool(BaseTool):
    name = "dbally"
    description: str
    collection: Collection
    args_schema: Type[BaseModel] = DatabaseQuery

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        """Use the tool synchronously."""
        try:
            result = asyncio.run(self.collection.ask(query))

            if result.textual_response is not None:
                return result.textual_response
            else:
                return result.results
        except UnsupportedQueryError:
            return "database master can't answer this question"

Now, let's test our tool. If everything goes correctly, you should see `[{'COUNT(*)': 10}]`. In case it doesn't, first make sure that provided `OPENAI KEY` is correct.

In [ ]:
from dbally.views.freeform.text2sql import configure_text2sql_auto_discovery, Text2SQLFreeformView
from dbally.llm_client.openai_client import OpenAIClient
import dbally

view_config = await configure_text2sql_auto_discovery(db_engine).discover()
recruitment_db = dbally.create_collection("recruitment", llm_client=OpenAIClient())
recruitment_db.add(Text2SQLFreeformView, lambda: Text2SQLFreeformView(db_engine, view_config))

DATABASE_TOOL = DballyTool(collection=recruitment_db, description="useful for when you need to gather some HR data")

In [ ]:
DATABASE_TOOL._run("How many job offers from Apple do we have?")

### State

Next, we define our agentic system's state as a typed dictionary containing an append-only list of messages. These messages form the chat history, which is all the state our simple assistant needs.

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph.message import AnyMessage, add_messages


class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

### Assistant Agent

Next, define the assistant agent. This simply takes the graph state and then calls an LLM for it to predict the best response. The most important thing is that we give access to the database tool to our assistant.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable
from langchain_openai import ChatOpenAI


class Assistant:
    def __init__(self, runnable: Runnable):
        self.runnable = runnable

    def __call__(self, state: State):
        result = self.runnable.invoke(state)
        return {"messages": result}


primary_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful talent aquisition assistant "
            " Use the provided tools to search for candidates, job offers, and other information to assist the user's queries. ",
        ),
        ("placeholder", "{messages}"),
    ]
)

assistant_llm = ChatOpenAI(model="gpt-3.5-turbo")
tools = [DATABASE_TOOL]
assistant_runnable = primary_assistant_prompt | assistant_llm.bind_tools(tools)
assistant_agent = Assistant(assistant_runnable)

Let's see how this agent works in separation. It is expected to see the Tool Call message generated.

In [ ]:
response = assistant_agent({"messages": ["Do we have any software engineers?"]})
response["messages"].pretty_print()

But, assistant doesn't know how to execute the tools. This is why we need to finish our system by connecting all building blocks to the graph

### Define Graph

Here we connect our previously generated agent by using [StateGraph](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.StateGraph), [ToolNode](https://langchain-ai.github.io/langgraph/reference/prebuilt/#toolnode), and [persistent memory](https://langchain-ai.github.io/langgraph/how-tos/persistence/) to build our final application

In [ ]:
from langgraph.graph import END, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.sqlite import SqliteSaver


tool_node = ToolNode(tools)

builder = StateGraph(State)
builder.add_node("assistant", assistant_agent)
builder.add_node("action", tool_node)
builder.set_entry_point("assistant")

builder.add_edge("action", "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
    # "action" calls one of our tools. END causes the graph to terminate (and respond to the user)
    {"action": "action", END: END},
)

memory = SqliteSaver.from_conn_string(":memory:")
graph = builder.compile(checkpointer=memory)

## Example conversation

Now it's time to try out our mighty chatbot!

In [ ]:
from uuid import uuid4

unique_id = uuid4().hex[0:8]

tutorial_questions = [
    "Hi do we have any software engineers?",
    "Describe me the first candidate, please.",
]

graph_config = {
    "configurable": {
        "thread_id": unique_id,
    }
}

for question in tutorial_questions:
    events = graph.stream({"messages": ("user", question)}, graph_config, stream_mode="values")
    for event in events:
        event["messages"][-1].pretty_print()

Congratulations! Together, we built an agentic system capable of querying the database. Good job!

The full code that you can just copy and paste to use is available in langgraph_tools.py